In [47]:
import xml.etree.cElementTree as ET
import bz2
import xml
import sys
import pprint
import pymongo
import re
from collections import defaultdict
from pymongo import MongoClient

In [38]:
seattleOSM = 'seattle_washington.osm'

In [39]:
with open(seattleOSM, 'r') as f:
    
    tagDict = {}
    subTags = {}
    maxLat = 0
    maxLon = 0
    
    for event, elem in ET.iterparse(f):
        if elem.tag in tagDict.keys():
            tagDict[elem.tag]['number'] += 1
            for attribute in elem.attrib:
                if attribute in tagDict[elem.tag].keys():
                    tagDict[elem.tag][attribute]+=1
                else:
                    tagDict[elem.tag][attribute] = 1
        else:
            tagNumber = {'number':1}
            tagDict[elem.tag] = tagNumber
            for attribute in elem.attrib:
                if attribute in tagDict[elem.tag].keys():
                    tagDict[elem.tag][attribute]+=1
                else:
                    tagDict[elem.tag][attribute] = 1
        elem.clear
    pprint.pprint(tagDict)

{'bounds': {'maxlat': 1, 'maxlon': 1, 'minlat': 1, 'minlon': 1, 'number': 1},
 'member': {'number': 40629, 'ref': 40629, 'role': 40629, 'type': 40629},
 'nd': {'number': 7320270, 'ref': 7320270},
 'node': {'changeset': 6620098,
          'id': 6620098,
          'lat': 6620098,
          'lon': 6620098,
          'number': 6620098,
          'timestamp': 6620098,
          'uid': 6619934,
          'user': 6619934,
          'version': 6620098},
 'osm': {'generator': 1, 'number': 1, 'timestamp': 1, 'version': 1},
 'relation': {'changeset': 6057,
              'id': 6057,
              'number': 6057,
              'timestamp': 6057,
              'uid': 6057,
              'user': 6057,
              'version': 6057},
 'tag': {'k': 4295818, 'number': 4295818, 'v': 4295818},
 'way': {'changeset': 632414,
         'id': 632414,
         'number': 632414,
         'timestamp': 632414,
         'uid': 632389,
         'user': 632389,
         'version': 632414}}


In [36]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

In [37]:
#client = MongoClient('localhost',27017)
#db = client['test-database']
#test_collection = db['seattle']

#clear seattle db before adding data
#db.seattle.remove({})

streetDict
with open(seattleOSM,'r') as f:
    i = 0
    for event,elem in ET.iterparse(f, events=("start",)):
        i+=1
        if i>10000:
            break
        if elem.tag == 'node' or elem.tag == 'way':    
            insertDict={}
            locationList = [0,0]
            tags = {}
            insertDict['type']=elem.tag
            for attributeKey in elem.keys():
                if attributeKey == 'tag':
                    pass
                elif attributeKey == 'lat':

                    locationList[0] = elem.attrib['lat']
                elif attributeKey == 'lon':
                    locationList[1] = elem.attrib['lon']
                else:
                    insertDict[attributeKey] = elem.attrib[attributeKey]
            for tag in elem.iter('tag'):
                try:
                    tagv = tag.attrib['v']
                    tagk = tag.attrib['k']
                    tags[tagk] = tagv
                    if is_street_name(tag):
                        print 'FOUND A STREET NAME------------------------------'
                        print tag.attrib
                except:
                    print 'FOUND A TAG ERROR -----------------'
                    print 'tag.attrib: ',tag.attrib
                    print 'tag.keys(): ',tag.keys()
            #If locationList isn't empty add it to the document
            if locationList != [0,0]:
                insertDict['location']=locationList
            if bool(tags):
                insertDict['tags'] = tags

            
            if 'tags' in insertDict.keys():
                if 'name' in insertDict['tags'].keys():
                    print'\nNew Dict\n'
                    pprint.pprint(insertDict)

            # UNCOMMENT TO WRITE TO DB !!!   
            #test_collection.insert_one(insertDict)
        else:
            pass
        elem.clear()
#db.collection_names(include_system_collections=False)


New Dict

{'changeset': '25678535',
 'id': '25883551',
 'location': ['48.4145385', '-123.3564537'],
 'tags': {'name': 'Cook Street Village', 'place': 'neighbourhood'},
 'timestamp': '2014-09-26T06:03:13Z',
 'type': 'node',
 'uid': '307202',
 'user': 'alester',
 'version': '5'}

New Dict

{'changeset': '25678535',
 'id': '25903927',
 'location': ['48.4613889', '-123.2970141'],
 'tags': {'name': 'Cadboro Bay Village',
          'place': 'neighbourhood',
          'website': 'http://www.cadborobayvillage.com/'},
 'timestamp': '2014-09-26T06:03:13Z',
 'type': 'node',
 'uid': '307202',
 'user': 'alester',
 'version': '5'}

New Dict

{'changeset': '14680500',
 'id': '26962238',
 'location': ['48.4601659', '-123.4428921'],
 'tags': {'name': 'Palmer (Fort Victoria)', 'railway': 'station'},
 'timestamp': '2013-01-17T06:49:41Z',
 'type': 'node',
 'uid': '307202',
 'user': 'alester',
 'version': '3'}

New Dict

{'changeset': '25678535',
 'id': '27047806',
 'location': ['48.4361598', '-123.339766

In [48]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave":"Avenue",
            "Rd.":"Road"
            }

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def auditStreetNames(osm_file):
    osm_file = open(osm_file, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
        elem.clear()
    return street_types

def update_name(name, mapping):
    print 'Name: ',name,'\nMapping: ',mapping
    mapKeys = mapping.keys()
    for keys in mapKeys:
        if keys in name:
            name = name[0:name.index(keys)]+mapping[keys]
    return name

def test():
    st_types = auditStreetNames(seattleOSM)
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
        
test()

{'1': set(['228th St SE Suite 1', 'Southeast 132nd Street #1']),
 '101': set(['156th Street East #101', 'East US Highway 101']),
 '104': set(['Northeast State Highway 104', 'State Highway 104']),
 '105': set(['State Route 105']),
 '110': set(['Northeast 4th Street, Suite 110']),
 '12': set(['HWY 12', 'State Route 12', 'US Highway 12']),
 '125': set(['Better Way SE Ste 125']),
 '140': set(['Highlands Dr NE Ste 140']),
 '149': set(['East Madison Street, Suite 149']),
 '14th': set(['North 14th']),
 '15th': set(['North 15th']),
 '16th': set(['North 16th']),
 '17th': set(['North 17th']),
 '18th': set(['North 18th', 'South 18th']),
 '19th': set(['19th', 'North 19th']),
 '2': set(['Highway 2', 'U.S. Hwy 2']),
 '20': set(['SR 20', 'Sr 20', 'State Route 20', 'West State Route 20']),
 '200': set(['4th Ave W, Suite 200']),
 '207': set(['West Republican Street Apt 207']),
 '20th': set(['South 20th']),
 '210': set(['224th St E # 210']),
 '214': set(['224th St E # 214']),
 '23rd': set(['North 23rd']

In [18]:
#Print first 100 items in seattle collection

seattle_1 = db.seattle
i=0
for entry in seattle_1.find():
    if i<100:
        pprint.pprint(entry) 
        i+=1
    else:
        break

{u'_id': ObjectId('55dbe48a942d8204d5cbff7f'),
 u'maxlat': u'48.528',
 u'maxlon': u'-121.335',
 u'minlat': u'46.608',
 u'minlon': u'-123.931',
 u'type': u'bounds'}
{u'_id': ObjectId('55dbe48a942d8204d5cbff80'),
 u'changeset': u'843918',
 u'id': u'21505268',
 u'lat': u'47.6812967',
 u'lon': u'-122.3212056',
 u'timestamp': u'2009-01-27T19:37:24Z',
 u'type': u'node',
 u'uid': u'34124',
 u'user': u'Sunny',
 u'version': u'3'}
{u'_id': ObjectId('55dbe48a942d8204d5cbff81'),
 u'changeset': u'9689672',
 u'id': u'25832758',
 u'lat': u'48.4364741',
 u'lon': u'-123.3129318',
 u'timestamp': u'2011-10-29T23:34:53Z',
 u'type': u'node',
 u'uid': u'6009',
 u'user': u'CoreyBurger',
 u'version': u'4'}
{u'_id': ObjectId('55dbe48a942d8204d5cbff82'),
 u'changeset': u'214775',
 u'id': u'25832786',
 u'lat': u'48.4350605',
 u'lon': u'-123.3129725',
 u'timestamp': u'2007-02-12T00:44:16Z',
 u'type': u'node',
 u'uid': u'6009',
 u'user': u'CoreyBurger',
 u'version': u'1'}
{u'_id': ObjectId('55dbe48a942d8204d5cbff8